- ticket_data.csv: Containing a ticket history (one line => a ticket proposal on
tictactrip)
- cities.csv the cities served by tictactrip (link thanks to the columns o_city (origin_city), d_city
(destination_city) from ticket_data)
- stations.csv stations served by tictactrip (link via o_station, d_station from ticket_data)
- providers.csv info on the different providers (link via company from ticket_data)
A provider is a "sub-company". For example TGV and TER are two providers of VSC
(SNCF trips)

In [1]:
import pandas as pd

C:\Users\npram\anaconda3\envs\tenpy\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\npram\anaconda3\envs\tenpy\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\npram\anaconda3\envs\tenpy\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
tickets = pd.read_csv("Data/ticket_data.csv")
cities = pd.read_csv("Data/cities.csv")
stations = pd.read_csv("Data/stations.csv")
providers = pd.read_csv("Data/providers.csv")

In [3]:
print(tickets.shape)
tickets.head(5)

(74168, 12)


,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


In [4]:
print(cities.shape)
cities.head(5)

(8040, 6)


,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN


In [5]:
print(stations.shape)
stations.head(5)

(11035, 4)


,id,unique_name,latitude,longitude
0,1,Aalen (Stuttgarter Straße),48.835296,10.092956
1,2,Aéroport Bordeaux-Mérignac,44.830226,-0.700883
2,3,Aéroport CDG,49.009900,2.559310
3,4,Aéroport de Berlin-Schönefeld,52.389446,13.520345
4,5,Aéroport de Dresden,51.123604,13.764737


In [6]:
print(providers.shape)
providers.head(5)

(227, 10)


,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
1,10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus
2,11,3,NaN,infobus,Infobus,False,False,False,False,bus
3,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus
4,13,5,NaN,flixbus,Flixbus,True,False,False,False,bus


In [7]:
import numpy as np
from sklearn.neighbors import DistanceMetric

stations['latitude'] = np.radians(stations['latitude'])
stations['longitude'] = np.radians(stations['longitude'])

dist = DistanceMetric.get_metric('haversine')

stations[["latitude", "longitude"]].to_numpy()
distance = (dist.pairwise(stations[["latitude", "longitude"]].to_numpy())*6373)
result = pd.DataFrame(distance,  columns=stations.unique_name.unique(), index=stations.unique_name.unique())
result.head(5)

,Aalen (Stuttgarter Straße),Aéroport Bordeaux-Mérignac,Aéroport CDG,Aéroport de Berlin-Schönefeld,Aéroport de Dresden,Aéroport de Genève,Aéroport de Paris Beauvais-Tillé,Aéroport de Prague (bus station),Aéroport de Tegel,Aéroport Marco Polo,...,Lerma,Lodz ZOB,Madrid - Avénida de America,Madrid - Gare routière Sud,Neum Konzum Supermarkt,Porto Avda De Los Aliados,Rom Ciampino Flughafen Piazzale Leonardo da Vinci,Sants,Vozilici D66,Wien Hotel ibis Vienna Airport
Aalen (Stuttgarter Straße),0.000000,933.178098,550.725094,463.349975,365.649019,416.389853,584.192913,333.326986,470.933676,407.399252,...,1317.340168,740.974499,1431.427012,1434.955655,877.246402,1695.141088,806.105565,1036.819953,512.449867,474.316978
Aéroport Bordeaux-Mérignac,933.178098,0.000000,526.643869,1337.885242,1281.139529,552.818809,557.502814,1266.316941,1335.240832,1024.175309,...,398.212231,1672.432355,545.769603,550.204748,1480.329666,762.400616,1126.053945,447.692560,1168.198887,1359.673257
Aéroport CDG,550.725094,526.643869,0.000000,857.666738,832.923114,407.876913,59.724620,852.757226,850.380249,833.950735,...,919.361149,1234.620430,1072.396445,1076.840812,1342.772487,1236.370706,1119.252655,849.318516,976.463112,1024.824510
Aéroport de Berlin-Schönefeld,463.349975,1337.885242,857.666738,0.000000,141.801093,869.806485,862.060753,259.114077,23.932406,770.530806,...,1734.095101,412.205296,1863.402410,1867.257880,1095.856606,2085.147805,1179.453174,1495.551222,805.822689,517.789244
Aéroport de Dresden,365.649019,1281.139529,832.923114,141.801093,0.000000,781.790102,847.091971,118.494447,162.354692,633.718120,...,1672.377640,401.707998,1792.694881,1796.339596,956.972318,2038.206083,1040.422869,1401.581196,664.308150,385.252863


# Goal 

* extract interesting information such as min, average and max price, min / max / average duration
per trip
* average price difference and duration depending on the train, bus and carpool depending on the distance from
route (0-200km, 201-800km, 800-2000km, 2000 + km)
* the most bonus info

In [12]:
cities_col = cities[["id", "unique_name"]]
new_df = pd.merge(left=tickets, right=cities_col, how='left', left_on="o_city", right_on="id")
new_df = new_df.rename(columns={"unique_name": "city_origin"})
new_df = new_df.drop("id_y", axis=1)

new_df = pd.merge(left=new_df, right=cities_col, how='left', left_on="d_city", right_on="id")
new_df = new_df.rename(columns={"unique_name": "city_destiny"})
new_df = new_df.drop("id", axis=1)

stations_col = stations[["id", "unique_name"]]
new_df = pd.merge(left=new_df, right=stations_col, how='left', left_on="o_station", right_on="id")
new_df = new_df.rename(columns={"unique_name": "station_origin"})
new_df = new_df.drop("id", axis=1)
new_df

new_df = pd.merge(left=new_df, right=stations_col, how='left', left_on="d_station", right_on="id")
new_df = new_df.rename(columns={"unique_name": "station_destiny"})
new_df = new_df.drop("id", axis=1)

providers_col = providers[["id", "name", "transport_type"]]
new_df = pd.merge(left=new_df, right=providers_col, how='left', left_on="company", right_on="id")
new_df = new_df.rename(columns={"name": "transport_name", "transport_type": "type"})
new_df = new_df.drop("id", axis=1)


In [13]:
new_df.to_csv("Data/data.csv", index= False)

EDA part

In [14]:
df = pd.read_csv("Data/data.csv")
print(df.shape)
df.tail(5)

(74168, 18)


,id_x,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city,city_origin,city_destiny,station_origin,station_destiny,transport_name,type
74163,6869182,13,279.0,10729.0,2017-10-27 02:30:00+00,2017-10-27 16:00:00+00,2590,2017-10-19 10:35:42.943+00,"{408,408}",{8371},628,562,paris,nantes,Gare de Bercy,Nantes Halluchère - Route de Paris,flixbus,bus
74164,6869185,9,279.0,304.0,2017-10-27 07:00:00+00,2017-10-27 13:30:00+00,2700,2017-10-19 10:35:42.943+00,"{1105,1105}",{9},628,562,paris,nantes,Gare de Bercy,Gare de Haluchère-Batignolles,ouibus,bus
74165,6869187,8371,10642.0,304.0,2017-10-27 08:30:00+00,2017-10-27 15:15:00+00,3000,2017-10-19 10:35:42.943+00,"{1105,1105}",{9},628,562,paris,nantes,Gare Routière Internationale de Paris-Gallieni...,Gare de Haluchère-Batignolles,isilines,bus
74166,6869189,13,279.0,304.0,2017-10-27 13:25:00+00,2017-10-27 21:10:00+00,2880,2017-10-19 10:35:42.943+00,"{863,863}",{13},628,562,paris,nantes,Gare de Bercy,Gare de Haluchère-Batignolles,flixbus,bus
74167,6869191,8371,10642.0,10729.0,2017-10-27 14:45:00+00,2017-10-27 21:40:00+00,2000,2017-10-19 10:35:42.943+00,"{863,863}",{8371},628,562,paris,nantes,Gare Routière Internationale de Paris-Gallieni...,Nantes Halluchère - Route de Paris,isilines,bus


In [16]:
transport = df.groupby(by="transport_name").nunique(dropna = True)
print("Number of distinct traveller companies: ", transport.shape[0])

Number of distinct traveller companies:  20


In [17]:
col_list = df["transport_name"].unique()

dicti = {}
for x in col_list:
    temp_data = df[df["transport_name"] == x]
    temp_data = temp_data["price_in_cents"].sort_values(ascending = False)
    df_max = temp_data.max()
    df_min = temp_data.min()
    df_avg = temp_data.mean()
    list_1 = [df_max,df_min,df_avg]
    dicti[x] = list_1

In [18]:
pd.DataFrame.from_dict(dicti, orient='index',columns=['Max_price', 'Low_price', 'Avg_price'])

,Max_price,Low_price,Avg_price
bbc,16150,300,2742.171907
ouibus,14596,850,3333.039045
corailintercite,27140,1140,6909.927041
coraillunea,13400,7530,10369.285714
flixbus,22900,850,3759.361631
isilines,13596,850,3075.794648
tgv,35100,1230,9099.485787
train,30160,490,6892.843413
eurolines,16890,1400,4747.602759
vatry,9600,5000,7062.500000


In [19]:
cols_list_type = df["type"].unique()
cols_list_type

array(['carpooling', 'bus', 'train'], dtype=object)

In [27]:
df = df.dropna()
df["distance"] = df.apply(lambda x: result[x["station_origin"]][x["station_destiny"]], axis=1)

In [36]:
temp = df[df["type"] == "bus"]
temp = temp.groupby(['station_origin','station_destiny',"type", "distance"]).size().sort_values(ascending=False)[:20]
ddf = pd.DataFrame(temp, columns=["count"])
ddf

count
station_origin                                     station_destiny                        type distance          
Gare SNCF Lyon-Perrache                            Gare Lille-Europe                      bus  559.109852     634
Gare Lille-Europe                                  Gare SNCF Lyon-Perrache                bus  559.109852     252
Gare de Bruxelles-Nord                             Parking Château Descas                 bus  764.198294     198
Gare de Bercy                                      Gare de Haluchère-Batignolles          bus  339.816308     150
Paris Nord - St-Denis université                   Gare Lille-Europe                      bus  195.042701     142
Aéroport Saint-Exupéry                             Gare de Bercy                          bus  402.190579     141
Gare de Bercy                                      Gare de Amsterdam Sloterdijk           bus  431.202876     130
                                                   Parking Château Descas                 bus  498.685922     129
Gare Routière Internationale de Paris-Gallieni ... Gare de Amsterdam Sloterdijk           bus  427.505660     124
Gare de Bercy                                      Gare Lille-Europe                      bus  206.373378     119
Gare SNCF Lyon-Perrache                            Gare routière de Caen                  bus  544.470606     111
La Défense                                         Gare Lille-Europe                      bus  203.397091      97
Gare SNCF Grenoble                                 Gare de Bercy                          bus  477.670977      94
Gare Routière Internationale de Paris-Gallieni ... Gare de Haluchère-Batignolles          bus  343.367123      93
Gare de Bercy                                      Gare routière Aéroport Nice Terminal 2 bus  684.756309      85
Gare routière de Perpignan                         Gare Lille-Europe                      bus  883.684151      83
Gare de Bercy                                      Gare SNCF Grenoble                     bus  477.670977      80
Paris Ibis Clichy Batignolles                      Gare Lille-Europe                      bus  201.831407      79
Gare de Bercy                                      Roma Tiburtina                         bus  1103.850572     78
Gare de Haluchère-Batignolles                      Gare de Bercy                          bus  339.816308      76

In [40]:
temp = df[df["type"] == "train"]
temp = temp.groupby(['station_origin','station_destiny',"type", "distance"]).size().sort_values(ascending=False)[:20]
ddf = pd.DataFrame(temp, columns=["count"])
ddf

count
station_origin                                     station_destiny                 type  distance         
Gare du Nord                                       Gare d'Amsterdam Centraal       train 428.176616    601
                                                   Gare Lille-Europe               train 202.222419    579
Gare de Bercy                                      Gare SNCF Lyon Part-Dieu        train 390.077602    409
Paris Montparnasse 1 Et 2                          Nantes                          train 339.262221    350
                                                   Gare SNCF Pau                   train 651.036593    244
Paris Gare de Lyon                                 Gare SNCF Lyon Part-Dieu        train 391.002542    243
Marne-La-Vallée - Chessy - Gare Tgv et OuiGo (À... Gare d'Amsterdam Centraal       train 417.856085    241
Collonges-au-Mont-d'Or                             Gare de Bercy                   train 382.766604    239
Paris Gare de Lyon                                 Gare de Marseille Saint-Charles train 658.344383    234
Gare SNCF Lyon Part-Dieu                           Gare de Bercy                   train 390.077602    220
Marne-La-Vallée - Chessy - Gare Tgv et OuiGo (À... Gare de Marseille Saint-Charles train 650.406097    218
Gare routière de Vannes                            Paris Montparnasse 1 Et 2       train 397.725969    200
Paris Montparnasse 1 Et 2                          Parking Château Descas          train 496.841088    199
Lyon Vaise                                         Gare de Bercy                   train 386.075562    189
Marne-La-Vallée - Chessy - Gare Tgv et OuiGo (À... Gare Lille-Europe               train 198.082946    174
Gare routière de Toulon                            Ventimiglia Stazione            train 154.414167    159
London St Pancras                                  Gare routière du Havre          train 227.421040    152
Gare de l'Est                                      Gare de Strasbourg              train 395.410637    138
Gare routière de Quimper                           Gare SNCF Béziers               train 768.441928    135
Gare de Marseille Saint-Charles                    Paris Gare de Lyon              train 658.344383    135

In [41]:
temp = df[df["type"] == "carpooling"]
temp = temp.groupby(['station_origin','station_destiny',"type", "distance"]).size().sort_values(ascending=False)[:20]
ddf = pd.DataFrame(temp, columns=["count"])
ddf

,,,,count
station_origin,station_destiny,type,distance,
